In [7]:
!pip install kaggle

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle datasets download -d amitanshjoshi/spotify-1million-tracks

Dataset URL: https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks
License(s): ODbL-1.0
 78% 60.0M/77.1M [00:00<00:00, 121MB/s]
100% 77.1M/77.1M [00:00<00:00, 139MB/s]


In [13]:
!unzip spotify-1million-tracks.zip -d /mnt/data/

Archive:  spotify-1million-tracks.zip
replace /mnt/data/spotify_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /mnt/data/spotify_data.csv  


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import DBSCAN
import numpy as np

In [2]:
tracks = pd.read_csv('/mnt/data/spotify_data.csv')
tracks['genre'] = tracks['genre'].str.strip().str.lower()
tracks.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
tracks.dropna()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.1390,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.5150,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.1450,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.5080,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.2170,171.864,244320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159759,1473391,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.3250,0.000141,0.1590,0.5220,107.951,344013,3
1159760,1473392,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.7880,0.000129,0.1300,0.2640,119.897,285067,4
1159761,1473393,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.4770,0.003130,0.0936,0.0351,100.076,214253,4
1159762,1473394,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.4310,0.000063,0.1250,0.2020,133.885,239133,3


In [ ]:
# features for clustering
features = ['danceability', 'energy', 'key', 'loudness', 'mode',
            'speechiness', 'acousticness', 'instrumentalness', 'liveness',
            'valence', 'tempo', 'duration_ms', 'time_signature']

# standardize feature cols
scaler = StandardScaler()
scaledTracks = scaler.fit_transform(tracks[features])

dbscan = DBSCAN(eps=0.5, min_samples=5)
tracks['cluster'] = dbscan.fit_predict(scaledTracks)

In [5]:
#GPT generated -> need to actually test
# Function to recommend tracks based on song name and artist name
def recommendSongs(inputTrackName, inputArtistName, tracks, features, scaler, dbscan, n_recommendations=10):
  inputTrackName = inputTrackName.strip().lower()
  inputArtistName = inputArtistName.strip().lower()

  # Find input track
  inputTrack = tracks[(tracks['track_name'].str.lower() == inputTrackName) & (tracks['artist_name'].str.lower() == inputArtistName)]

  if inputTrack.empty:
    print("Track not found in dataset")
    return inputTrack
  print("Input track found:")
  print(inputTrack)

  inputTrackFeatures = inputTrack[features]
  scaledInputTrackFeatures = scaler.transform(inputTrackFeatures)

  # Get the cluster of the input track
  inputCluster = inputTrack['cluster'].values[0]
  print(f"Predicted cluster: {inputCluster}")

  if inputCluster == -1:
    print("The input track is considered noise by DBSCAN and does not belong to any cluster.")
    return pd.DataFrame()

  # Get tracks from the same cluster
  similarTracks = tracks[tracks['cluster'] == inputCluster]

  # Exclude input track from recommendations
  similarTracks = similarTracks[(similarTracks['track_name'].str.lower() != inputTrackName) | (similarTracks['artist_name'].str.lower() != inputArtistName)]

  # Calculate the distances to input track
  similarTracksFeatures = scaler.transform(similarTracks[features])
  distances = euclidean_distances(scaledInputTrackFeatures, similarTracksFeatures).flatten()

  # Add distances to the DataFrame
  similarTracks['distance'] = distances

  # Sort by distance and select top 10
  recommendations = similarTracks.sort_values(by=['distance', 'popularity'], ascending=[True, False]).drop_duplicates(subset=['artist_name', 'track_name']).head(n_recommendations)

  if len(recommendations) < n_recommendations:
    # Fill remaining recommendations with similar tracks if duplicates
    remaining = similarTracks[~similarTracks.index.isin(recommendations.index)].head(n_recommendations - len(recommendations))
    recommendations = pd.concat([recommendations, remaining])

  return recommendations[['artist_name', 'track_name', 'popularity', 'genre', 'year']]

  # Get user input
  inputTrackName = input("Enter track name: ")
  inputArtistName = input("Enter artist name: ")

  # Get recommendations
  recommendations = recommendSongs(inputTrackName, inputArtistName, tracks, selected_features, scaler, dbscan)

  if recommendations.empty:
    print("No recommendations found")
  else:
    print("Here are the top recommendations based on the inputted song:")
    print(recommendations[['artist_name', 'track_name', 'popularity', 'genre', 'year']])
